In [1]:
# Импорт необходимых библиотек
import os

import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [2]:
# Чтение файла
with open('text', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [3]:
# Укажем сколько может быть слов в словаре, а также разобьем текст на отдельные слова
maxWordsCount = 1500
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
tokenizer.word_index

{'не': 1,
 'и': 2,
 'в': 3,
 'вы': 4,
 'на': 5,
 'я': 6,
 'только': 7,
 'а': 8,
 'все': 9,
 'что': 10,
 'это': 11,
 'если': 12,
 'жизнь': 13,
 'просто': 14,
 'с': 15,
 'никогда': 16,
 'чем': 17,
 'чтобы': 18,
 'у': 19,
 'к': 20,
 'то': 21,
 'себя': 22,
 'больше': 23,
 'всегда': 24,
 'меня': 25,
 'за': 26,
 'вас': 27,
 'мне': 28,
 'от': 29,
 'когда': 30,
 'вам': 31,
 'никому': 32,
 'сделать': 33,
 'есть': 34,
 'жизни': 35,
 'без': 36,
 'быть': 37,
 'но': 38,
 'позитивным': 39,
 'такой': 40,
 'тебя': 41,
 'как': 42,
 'никто': 43,
 'для': 44,
 'много': 45,
 'тем': 46,
 'деньги': 47,
 'своих': 48,
 'денег': 49,
 'ты': 50,
 'день': 51,
 'позитивное': 52,
 'мы': 53,
 'ничего': 54,
 'значит': 55,
 'лучше': 56,
 'надо': 57,
 'так': 58,
 'свою': 59,
 'них': 60,
 'того': 61,
 'позитивно': 62,
 'которые': 63,
 'мечты': 64,
 'нужно': 65,
 'тебе': 66,
 'опасно': 67,
 'равно': 68,
 'ни': 69,
 'нет': 70,
 'успеха': 71,
 'же': 72,
 'сама': 73,
 'стоит': 74,
 'болезней': 75,
 'энергию': 76,
 'себе': 77

In [4]:
# Выведем некоторые слова и их частотность употребления
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('я', 21), ('притягиваю', 1), ('только', 21), ('плохое', 1), ('кому', 2), ('нужен', 2), ('с', 12), ('такой', 5), ('внешностью', 2), ('не', 91)]


In [5]:
# Преобразуем текст в последовательность чисел в соответствие со словарем
data = tokenizer.texts_to_sequences([texts])
res = to_categorical(data[0], num_classes=maxWordsCount)
print(data)
print(res.shape)

[[6, 197, 7, 198, 102, 6, 103, 15, 40, 104, 1, 105, 32, 41, 199, 200, 12, 201, 202, 21, 203, 11, 1, 204, 205, 6, 66, 1, 106, 32, 9, 206, 2, 207, 11, 67, 18, 6, 1, 208, 28, 9, 68, 1, 209, 6, 1, 107, 15, 210, 54, 211, 42, 108, 6, 1, 212, 29, 213, 1, 214, 28, 1, 215, 216, 3, 217, 218, 55, 2, 219, 6, 220, 19, 25, 109, 1, 221, 16, 6, 24, 9, 222, 110, 40, 223, 28, 9, 68, 1, 224, 225, 111, 1, 226, 102, 6, 112, 227, 228, 28, 24, 1, 229, 25, 43, 1, 113, 19, 25, 114, 230, 6, 231, 28, 232, 233, 10, 4, 11, 109, 1, 234, 19, 25, 112, 114, 235, 18, 6, 1, 236, 69, 3, 17, 70, 71, 115, 237, 1, 238, 33, 116, 6, 16, 1, 239, 26, 240, 11, 72, 241, 67, 6, 32, 1, 242, 11, 67, 7, 243, 117, 72, 244, 118, 245, 246, 16, 1, 247, 248, 56, 2, 1, 249, 28, 16, 1, 250, 251, 252, 1, 253, 28, 119, 254, 6, 73, 255, 5, 22, 256, 257, 16, 1, 258, 111, 1, 74, 2, 259, 10, 260, 29, 118, 261, 7, 262, 120, 15, 263, 264, 7, 265, 2, 266, 20, 267, 7, 268, 269, 270, 271, 75, 69, 121, 272, 1, 273, 274, 275, 276, 29, 277, 75, 278, 43, 

In [6]:
data = tokenizer.texts_to_matrix('все') #последовательности токенов в OHE
print(data)
word_tokens = tokenizer.texts_to_matrix('все').argmax(axis=1)
word_tokens

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


array([ 3, 15,  0], dtype=int64)

In [7]:
# Настроим обучающую выборку
inp_words = 3
n = res.shape[0] - inp_words
n

1664

In [8]:
# Сформируем трехмерный тензор
X = np.array([res[i:i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [9]:
# Построим RNN
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               208512    
                                                                 
 dense (Dense)               (None, 1500)              193500    
                                                                 
Total params: 402,012
Trainable params: 402,012
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Скомпилируем модель и обучим модель
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)
model.evaluate(X, Y)


Epoch 1/50
52/52 [==============================] - 1s 3ms/step - loss: 7.1836 - accuracy: 0.0409
Epoch 2/50
52/52 [==============================] - 0s 3ms/step - loss: 6.2560 - accuracy: 0.0547
Epoch 3/50
52/52 [==============================] - 0s 3ms/step - loss: 6.0081 - accuracy: 0.0547
Epoch 4/50
52/52 [==============================] - 0s 3ms/step - loss: 5.8539 - accuracy: 0.0583
Epoch 5/50
52/52 [==============================] - 0s 3ms/step - loss: 5.7014 - accuracy: 0.0625
Epoch 6/50
52/52 [==============================] - 0s 3ms/step - loss: 5.4848 - accuracy: 0.0793
Epoch 7/50
52/52 [==============================] - 0s 3ms/step - loss: 5.1797 - accuracy: 0.1064
Epoch 8/50
52/52 [==============================] - 0s 3ms/step - loss: 4.7916 - accuracy: 0.1520
Epoch 9/50
52/52 [==============================] - 0s 3ms/step - loss: 4.3196 - accuracy: 0.2188
Epoch 10/50
52/52 [==============================] - 0s 3ms/step - loss: 3.8199 - accuracy: 0.3389
Epoch 11/50
52/52 [

[0.11475104838609695, 0.9885817170143127]

In [11]:
# Добавим функцию для построения фраз
def buildPhrase(texts, str_len=5):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [12]:
# Проверим модель
res = buildPhrase("Худшие времена могут")
print(res)
# Проверим модель
res = buildPhrase("Чтобы создать радугу")
print(res)

1/1 [==============================] - 0s 15ms/step
Худшие времена могут стать лучшими если вы думаете
1/1 [==============================] - 0s 16ms/step
Чтобы создать радугу нужны солнечный свет и дождь


In [13]:
# Проверим модель
res = buildPhrase("Только хорошее действие")
print(res)
# Проверим модель
res = buildPhrase("Позитивное мышление и")
print(res)

1/1 [==============================] - 0s 13ms/step
Только хорошее действие в сочетании с позитивным мышлением
1/1 [==============================] - 0s 15ms/step
Позитивное мышление и визуализация моего успеха были моими


Используем RNN со слоем Embedding

In [14]:
# Подготовим данные
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )

In [15]:
# Заново сформируем трехмерный тензор
X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [16]:
# Построим RNN
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 256)            384000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_1 (Dense)             (None, 1500)              193500    
                                                                 
Total params: 626,780
Trainable params: 626,780
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Скомпилируем модель и обучим модель
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)
model.evaluate(X, Y)

Epoch 1/50
52/52 [==============================] - 1s 4ms/step - loss: 7.1857 - accuracy: 0.0373
Epoch 2/50
52/52 [==============================] - 0s 4ms/step - loss: 6.2511 - accuracy: 0.0547
Epoch 3/50
52/52 [==============================] - 0s 4ms/step - loss: 5.8644 - accuracy: 0.0583
Epoch 4/50
52/52 [==============================] - 0s 4ms/step - loss: 5.4831 - accuracy: 0.0757
Epoch 5/50
52/52 [==============================] - 0s 4ms/step - loss: 4.9617 - accuracy: 0.1256
Epoch 6/50
52/52 [==============================] - 0s 4ms/step - loss: 4.3508 - accuracy: 0.2175
Epoch 7/50
52/52 [==============================] - 0s 4ms/step - loss: 3.7395 - accuracy: 0.3486
Epoch 8/50
52/52 [==============================] - 0s 4ms/step - loss: 3.1448 - accuracy: 0.4850
Epoch 9/50
52/52 [==============================] - 0s 4ms/step - loss: 2.6109 - accuracy: 0.5944
Epoch 10/50
52/52 [==============================] - 0s 4ms/step - loss: 2.1456 - accuracy: 0.6899
Epoch 11/50
52/52 [

[0.030275123193860054, 0.9915865659713745]

In [18]:
# Изменим функцию для построения фраз
def buildPhrase(texts, str_len=20):
        res = texts
        data = tokenizer.texts_to_sequences([texts])[0]
        for i in range(str_len):
            x = data[i: i + inp_words]
            inp = np.expand_dims(x, axis=0)

            pred = model.predict(inp)
            indx = pred.argmax(axis=1)[0]
            data.append(indx)

            res += " " + tokenizer.index_word[indx]  # дописываем строку

        return res

In [19]:
# Проверим модель
res = buildPhrase("Худшие времена могут")
print(res)
# Проверим модель
res = buildPhrase("Чтобы создать радугу")
print(res)

1/1 [==============================] - 0s 14ms/step
Худшие времена могут стать лучшими если вы думаете о них с хорошим настроем и пытайся улучшиться позитивное всё лучше чем негативное ничего никто
1/1 [==============================] - 0s 14ms/step
Чтобы создать радугу нужны солнечный свет и дождь а без них не было бы радуги худшие времена могут стать лучшими если вы думаете


In [20]:
# Проверим модель
res = buildPhrase("Только хорошее действие")
print(res)
# Проверим модель
res = buildPhrase("Позитивное мышление и")
print(res)

1/1 [==============================] - 0s 15ms/step
Только хорошее действие в сочетании с позитивным мышлением приводит к успеху чтобы создать радугу нужны солнечный свет и дождь а без них не
1/1 [==============================] - 0s 14ms/step
Позитивное мышление и визуализация моего успеха были моими ключами к успеху только хорошее действие в сочетании с позитивным мышлением приводит к успеху чтобы
